In [ ]:
from geodeticengine import CoordTrans, CrsSearch, CtSearch
import json

## Transformation examples

### Transforming coordinates from an engineering CRS to a global CRS and back to engineering again

In [ ]:
points = [[8911.832,5139.165]]
crs_from = "EQUINOR:4100002" # ST_MON_Refinery_E4100002
crs_to = "EPSG:25832"        # ST_ETRS89_UTM32N_P25832
ct_from = "EQUINOR:3100002"  # ST_MON_ETRS89_P25832_T3100002
ct = CoordTrans(crs_from=crs_from, crs_to=crs_to, ct_from=ct_from, points=points)
transformed_coords = ct.transform_pointlist()
print(f"Coordinates transformed to {crs_to}:\n{transformed_coords}")

# Transform back to engineering
ct = CoordTrans(crs_from=crs_to, crs_to=crs_from, ct_to=ct_from, points=transformed_coords)
transformed_coords_back = ct.transform_pointlist()
print(f"Coordinates transformed back to the engineering CRS {crs_to}:\n{transformed_coords_back}")

### Transform coordinates by specifying EPSG codes for both the source and target CRS and the transformation between them and by sending the coordinates to the API.

In [ ]:
points = [[10, 60]]
crs_from = "EPSG:4230"
crs_to = "EPSG:4326"
ct_from = "EPSG:1612"

# Transform coordinates
ct = CoordTrans(crs_from=crs_from, crs_to=crs_to, ct_from=ct_from, points=points)
print(f"Transformed coordinates:{ct.transform_pointlist()}")

# Get transformation pipeline
pipeline = ct.get_pipeline()
print(f"Transformation pipeline: {pipeline}")

### Transform coordinates using a bound CRS and by sending the coordinates to the API.

In [ ]:
points = [[9,65],[12,70]]
crs_from = "ST_ED50_T1133"
crs_to = "ST_WGS84_G4326"

# Transform coordinates
ct = CoordTrans(crs_from=crs_from, crs_to=crs_to, points=points)
print(f"Transformed coordinates:{ct.transform_pointlist()}")

# Get transformation pipeline
pipeline =  ct.get_pipeline()
print(f"Transformation pipeline: {pipeline}")


### Transform coordinates without sending them to the API. Only the transformation pipeline is extracted from the API and the transformation is done ``locally`` using pyproj 

In [ ]:
crs_from = "EPSG:32632" # WGS84 / UTM zone 32N
crs_to = "EPSG:23031"   # ED50 / UTM zone 31N
ct_code = "EPSG:1613"   # Transformation from  ED50 til WGS84

points = [[273178.571, 6846980.063]]

# Create a transformer object of the source crs, target crs and the transformation between them
trans_obj = CoordTrans(crs_from, crs_to, ct_from=ct_code)

# Get the transformation pipeline
trans_pipeline = trans_obj.get_pipeline()

# Use the transformation pipeline extracted in the line above to transform the coordiantes
transformed_coord_lst = trans_obj.transform_from_pipeline(points, trans_pipeline)
print(f"Transformed coordinates by using the pipeline from the GeodeticEngine:\n{transformed_coord_lst}")

### Transform coordiantes by using two different transformations (two-step)
First transform the coordinates from ``ETRS89`` to ``WGS84`` using the ``EQUINOR:3000034`` transformation, and then transform from ``WGS84`` to ``ED50`` using ``EPSG:1613``

``NB: Pyproj is in this case only used to get a pretty formatting on the transformation pipeline.``

In [ ]:
from pyproj import Transformer

crs_from = "EPSG:25832"          # ETS89 / UTM zone 32N
crs_to = "EPSG:23031"            # ED50 / UTM zone 31N
ct_from_code = "EQUINOR:3000034" # Equinor custom transformation from ETRS89 to WGS84
ct_to_code = "EPSG:1613"         # Transformation from ED50 to WGS84

points = [[273178.571, 6846980.063]]

# Create a transformer object of the source crs, target crs and the transformation between them
trans_obj = CoordTrans(crs_from, crs_to, ct_from=ct_from_code, ct_to=ct_to_code)

# Get the transformation pipeline
trans_pipeline = trans_obj.get_pipeline()
print(f"The transformation pipeline for the two transformations:\n{Transformer.from_pipeline(trans_pipeline).to_proj4(pretty=True)}\n")

# Use the transformation pipeline extracted in the line above to transform the coordiantes
transformed_coord_lst = trans_obj.transform_from_pipeline(points, trans_pipeline)
print(f"Transformed coordinates by using the pipeline from the GeodeticEngine:\n{transformed_coord_lst}")

### Convert geographic coordinates (lat/lon) to projected coordinates

In [ ]:
from geodeticengine import CoordTrans

crs_from = "EPSG:4326"           # WGS84 geograpich 2D (lat/lon)
crs_to = "EPSG:32631"            # WGS84 / UTM zone 31N

points = [[1.97926020, 60.53835841]] # longitude first, then latitude (degree)

# Create a transformer object of the source crs, target crs
trans_obj = CoordTrans(crs_from, crs_to)

# Get the transformation pipeline
conv_pipeline = trans_obj.get_pipeline()
print(f"The pipeline for the conversion:\n{trans_pipeline}\n")

# Use the transformation pipeline extracted in the line above to convert the coordinates
converted_coord_lst = trans_obj.transform_from_pipeline(points, conv_pipeline)
print(f"Converted coordinates by using the pipeline from the GeodeticEngine:\n{converted_coord_lst}")

## Listing examples

#### Get all bound CRS to WGS84

In [ ]:

# Get all bound CRS to WGS84
crs_query = CrsSearch(types=["bound projected","bound geographic 2D"], target_crs="ST_WGS84_G4326")
all_bound_crs = json.dumps(crs_query.get_entities(), indent=2)
print(f"All bound CRS:\n{all_bound_crs}")


#### Get all bound Projeced and Projected CRS

In [ ]:
crs_query = CrsSearch(types=["bound projected","projected"])
all_bound_proj_and_proj = json.dumps(crs_query.get_entities(), indent=2)
print(f'All "Bound Projeced" and "Projected":\n{all_bound_proj_and_proj}')

#### Get all transformations between ED50 and WGS 84 

In [ ]:
ct_query = CtSearch(types=["transformation","concatenated operation"], source_crs="ST_ED50_G4230", target_crs="ST_WGS84_G4326")
cts = json.dumps(ct_query.get_entities(), indent=2)
print(f'All transformation between ED50 and WGS 84:\n{cts}')


### List all available transformations between ED50 and WGS84


In [ ]:
ct_query = CtSearch(types=["transformation","concatenated operation"], source_crs="ST_ED50_G4230", target_crs="ST_WGS84_G4326")
all_available_trans = json.dumps(ct_query.get_entities(), indent=2)
print(f'All availiable transformation between ED50 and WGS 84 in total:\n{all_available_trans}')